# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 New York Times Comments

1. 데이터 전처리

In [2]:
import numpy as np
import pandas as pd
from string import punctuation

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
df=pd.read_csv('../dataset/ArticlesMay2017.csv', encoding = "latin")
df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,5906b3197c459f24986dd003,2412,By IAN JOHNSON,article,My Beijing: The Sacred City,"['Travel and Vacations', 'Beijing (China)', 'R...",3,Travel,1,2017-05-01 04:01:21,Unknown,"This metropolis was once a total work of art, ...",The New York Times,News,https://www.nytimes.com/2017/05/01/travel/beij...
1,NaN,5906b3297c459f24986dd004,2318,By EMMA G. FITZSIMMONS,article,"6 Million Riders a Day, 1930s Technology","['Subways', 'Delays (Transportation)', 'Transi...",68,Metro,1,2017-05-01 04:01:33,Unknown,New Yorkâs subway is struggling with old inf...,The New York Times,News,https://www.nytimes.com/2017/05/01/nyregion/ne...
2,NaN,5906ceec7c459f24986dd021,1795,By MARC TRACY,article,Seeking a Cross-Border Conference,"['Cetys University', 'College Athletics', 'Nat...",3,Sports,1,2017-05-01 06:00:05,Unknown,Cetys University is making an ambitious bid to...,The New York Times,News,https://www.nytimes.com/2017/05/01/sports/mexi...
3,NaN,5906cfa77c459f24986dd022,213,By SHANNON DOYNE,article,"Questions for: âDespite the âYuck Factor,â...",[],3,Learning,0,2017-05-01 06:03:03,Unknown,How are leeches used to treat various medical ...,The New York Times,News,https://www.nytimes.com/2017/05/01/learning/qu...
4,NaN,5906e1c07c459f24986dd039,1342,By JASON STANLEY,article,Who Is a âCriminalâ?,"['Illegal Immigration', 'Traffic and Parking V...",3,OpEd,0,2017-05-01 07:20:26,Unknown,Justice Roberts was right. The Trump administr...,The New York Times,Op-Ed,https://www.nytimes.com/2017/05/01/opinion/who...


In [4]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  16
Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [5]:
# headline 열의 데이터만 사용, Null 검사
df['headline'].isnull().values.any()

False

In [6]:
# headline 열의 데이터로 리스트 생성
headline = [title for title in df.headline.values]
headline[:5]

['My Beijing: The Sacred City',
 '6 Million Riders a Day, 1930s Technology',
 'Seeking a Cross-Border Conference',
 'Questions for: â\x80\x98Despite the â\x80\x9cYuck Factor,â\x80\x9d Leeches Are Big in Russian Medicineâ\x80\x99',
 'Who Is a â\x80\x98Criminalâ\x80\x99?']

In [7]:
len(headline)

996

In [8]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

935

In [9]:
# 구두점 제거와 소문자화를 위한 함수
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [10]:
text = [repreprocessing(x) for x in headline]
text[:5]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal']

In [11]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 2653


In [12]:
sequences = []
for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[48, 799],
 [48, 799, 1],
 [48, 799, 1, 800],
 [48, 799, 1, 800, 57],
 [114, 406],
 [114, 406, 407],
 [114, 406, 407, 2],
 [114, 406, 407, 2, 179],
 [114, 406, 407, 2, 179, 801],
 [114, 406, 407, 2, 179, 801, 802],
 [803, 2]]

In [13]:

index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 :', index_to_word[1])
print('빈도수 상위 582번 단어 :', index_to_word[582])

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : taiwan


In [14]:
max_len=max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 16


In [15]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 799]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  48 799   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0  48 799   1 800]]


In [16]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [17]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [18]:
X.shape, y.shape

((5501, 15), (5501, 2653))

# 2. 모델 설계 및 학습

In [19]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [20]:

# 임베딩 벡터는 10차원, 은닉 상태 크기는 128
embedding = Embedding(vocab_size, 10, input_length=max_len-1, name="Embedding_Layer")
lstm = LSTM(128, name="LSTM_Layer")
output = Dense(vocab_size, activation='softmax', name="Output_Layer")

In [21]:

model = Sequential()
model.add(embedding)
model.add(lstm)
model.add(output)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 15, 10)            26530     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 2653)              342237    
Total params: 439,935
Trainable params: 439,935
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [23]:
history = model.fit(X, y, epochs=200, verbose=2)

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
 - 2s - loss: 7.5198 - accuracy: 0.0302
Epoch 2/200
 - 2s - loss: 6.9990 - accuracy: 0.0369
Epoch 3/200
 - 2s - loss: 6.8693 - accuracy: 0.0371
Epoch 4/200
 - 2s - loss: 6.7764 - accuracy: 0.0384
Epoch 5/200
 - 2s - loss: 6.6698 - accuracy: 0.0460
Epoch 6/200
 - 2s - loss: 6.5328 - accuracy: 0.0489
Epoch 7/200
 - 2s - loss: 6.3744 - accuracy: 0.0534
Epoch 8/200
 - 2s - loss: 6.1969 - accuracy: 0.0558
Epoch 9/200
 - 2s - loss: 6.0103 - accuracy: 0.0573
Epoch 10/200
 - 2s - loss: 5.8230 - accuracy: 0.0649
Epoch 11/200
 - 2s - loss: 5.6446 - accuracy: 0.0713
Epoch 12/200
 - 2s - loss: 5.4785 - accuracy: 0.0769
Epoch 13/200
 - 2s - loss: 5.3220 - accuracy: 0.0793
Epoch 14/200
 - 2s - loss: 5.1718 - accuracy: 0.0856
Epoch 15/200
 - 2s - loss: 5.0304 - accuracy: 0.0923
Epoch 16/200
 - 2s - loss: 4.8935 - accuracy: 0.1038
Epoch 17/200
 - 2s - loss: 4.7653 - accuracy: 0.1122
Epoch 18/200
 - 2s - loss: 4.6397 - accuracy: 0.1214
Epoch 19/200
 - 2s - loss: 4.5229 - accuracy: 0.1349
Ep

Epoch 155/200
 - 2s - loss: 0.3866 - accuracy: 0.8933
Epoch 156/200
 - 2s - loss: 0.3823 - accuracy: 0.8933
Epoch 157/200
 - 2s - loss: 0.3817 - accuracy: 0.8949
Epoch 158/200
 - 2s - loss: 0.3773 - accuracy: 0.8937
Epoch 159/200
 - 2s - loss: 0.3757 - accuracy: 0.8927
Epoch 160/200
 - 2s - loss: 0.3731 - accuracy: 0.8962
Epoch 161/200
 - 2s - loss: 0.3862 - accuracy: 0.8917
Epoch 162/200
 - 2s - loss: 0.4349 - accuracy: 0.8829
Epoch 163/200
 - 2s - loss: 0.3850 - accuracy: 0.8922
Epoch 164/200
 - 2s - loss: 0.3691 - accuracy: 0.8947
Epoch 165/200
 - 2s - loss: 0.3628 - accuracy: 0.8960
Epoch 166/200
 - 2s - loss: 0.3615 - accuracy: 0.8964
Epoch 167/200
 - 2s - loss: 0.3587 - accuracy: 0.8951
Epoch 168/200
 - 2s - loss: 0.3572 - accuracy: 0.8951
Epoch 169/200
 - 2s - loss: 0.3559 - accuracy: 0.8940
Epoch 170/200
 - 2s - loss: 0.3542 - accuracy: 0.8949
Epoch 171/200
 - 2s - loss: 0.3527 - accuracy: 0.8953
Epoch 172/200
 - 2s - loss: 0.3517 - accuracy: 0.8955
Epoch 173/200
 - 2s - loss: 

In [26]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=15, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [98]:
text = [repreprocessing(x) for x in headline]
#text[487:488] i accidentally killed a child may i contact the family clock
#text[706:707]
text
#t.texts_to_sequences(['how to be mindful while swimming'])[0]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal',
 'an antidote to europes populism',
 'the cost of a speech',
 'degradation of the language',
 'on the power of being awful',
 'trump garbles pitch on a revised health bill',
 'whats going on in this picture  may 1 2017',
 'when patients hit a medical wall',
 'for pregnant women getting serious about whooping cough',
 'new york city transit reporter in wonderland riding the london tube',
 'how to cut an avocado without cutting yourself',
 'in fictional suicide health experts say they see a real cause for alarm',
 'claims of liberal media bias hit espn too',
 'is the dream in australia crumbling',
 'police in texas change account in officers fatal shooting of 15yearold',
 'most adults favor sex ed most students dont get it',
 'australia feels its ties to us put it in a bind',
 'a

In [107]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'how', 8))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성
#print(sentence_generation(model, t, 'former', 2))

i accidentally killed a child may i contact the family clock
how to be mindful while swimming knee mother rights
